# Structured Generation for Information Retrieval

In this notebook, we explore structured generation techniques using Large Language Models (LLMs) to extract and organize information from financial documents efficiently. Structured generation allows us to transform unstructured text into structured, queryable data, making it easier to analyze and utilize information in financial workflows.

## Key Topics

- Fundamentals of structured generation with LLMs
- JSON and XML output formatting techniques
- Schema design for financial information extraction
- Validation and error handling in structured outputs
- Advanced patterns for complex information retrieval
- Practical applications in financial document processing

## 1. Setup and Dependencies

In [ ]:
# Install required packages
!pip install openai pandas numpy matplotlib plotly pydantic jsonschema anthropic langchain langchain_openai

# Import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import re
from pydantic import BaseModel, Field, validator
from typing import List, Dict, Optional, Union
import jsonschema
from openai import OpenAI
import anthropic
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import warnings
warnings.filterwarnings('ignore')

# Initialize API clients
openai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "your-api-key-here"))
anthropic_client = anthropic.Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY", "your-api-key-here"))

# Set random seed for reproducibility
np.random.seed(42)

## 2. Understanding Structured Generation

Structured generation refers to the technique of instructing LLMs to produce outputs in a specific format that follows a predefined structure. This is particularly valuable in financial applications where precise data extraction and organization are crucial.

### Why Structured Generation Matters in Finance:

1. **Data Consistency**: Ensures information is extracted in a consistent, standardized format
2. **Automated Processing**: Enables direct integration with downstream systems and databases
3. **Error Reduction**: Minimizes manual intervention and associated errors
4. **Scalability**: Allows processing large volumes of financial documents efficiently
5. **Information Retrieval**: Makes complex financial information easily queryable and searchable

Let's start by looking at the difference between unstructured and structured outputs:

In [ ]:
# Example of unstructured vs structured prompting
basic_prompt = """
Extract the following information from this earnings call transcript:
- Company name
- Quarter and year
- Revenue
- Net income
- EPS
- Key highlights

Transcript:
"Welcome to the Q2 2023 earnings call for TechInvest Corp. I'm pleased to report another strong quarter with revenue reaching $2.4 billion, representing a 15% year-over-year growth. Our net income was $340 million, resulting in an EPS of $1.85, exceeding analyst expectations of $1.70. Key highlights for the quarter include the launch of our new AI-powered trading platform, expansion into three new markets, and a 22% increase in our institutional client base."
"""

structured_prompt = """
Extract the following information from this earnings call transcript and format it as JSON:

Transcript:
"Welcome to the Q2 2023 earnings call for TechInvest Corp. I'm pleased to report another strong quarter with revenue reaching $2.4 billion, representing a 15% year-over-year growth. Our net income was $340 million, resulting in an EPS of $1.85, exceeding analyst expectations of $1.70. Key highlights for the quarter include the launch of our new AI-powered trading platform, expansion into three new markets, and a 22% increase in our institutional client base."

Output the information in the following JSON format:
{
  "company_name": "string",
  "quarter": "string",
  "year": "number",
  "financial_metrics": {
    "revenue": {
      "value": "number",
      "unit": "string"
    },
    "net_income": {
      "value": "number",
      "unit": "string"
    },
    "eps": {
      "value": "number",
      "unit": "string"
    }
  },
  "key_highlights": ["string"]
}
"""

# Call the model with both prompts
response_unstructured = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": basic_prompt}]
)

response_structured = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": structured_prompt}]
)

# Print results
print("UNSTRUCTURED OUTPUT:")
print("-" * 50)
print(response_unstructured.choices[0].message.content)

print("\nSTRUCTURED OUTPUT:")
print("-" * 50)
print(response_structured.choices[0].message.content)

# Parse the structured output to demonstrate its machine-readability
try:
    structured_data = json.loads(response_structured.choices[0].message.content)
    print("\nPARSED JSON DATA:")
    print("-" * 50)
    print(f"Company: {structured_data['company_name']}")
    print(f"Period: Q{structured_data['quarter']} {structured_data['year']}")
    print(f"Revenue: ${structured_data['financial_metrics']['revenue']['value']} {structured_data['financial_metrics']['revenue']['unit']}")
    print(f"Net Income: ${structured_data['financial_metrics']['net_income']['value']} {structured_data['financial_metrics']['net_income']['unit']}")
    print(f"EPS: ${structured_data['financial_metrics']['eps']['value']}")
    print("Key Highlights:")
    for highlight in structured_data['key_highlights']:
        print(f"- {highlight}")
except json.JSONDecodeError:
    print("Could not parse the JSON response")

## 3. Schema Design for Financial Information

One of the most important aspects of structured generation is designing effective schemas. A well-designed schema ensures that the LLM extracts precisely the information you need in the format you require.

### Schema Design Principles:

1. **Clarity**: Define fields with clear, unambiguous names
2. **Completeness**: Include all necessary information fields
3. **Hierarchy**: Organize related information in nested structures
4. **Typing**: Specify data types for each field
5. **Validation Rules**: Include constraints where appropriate

Let's design schemas for common financial document types:

In [ ]:
# Define Pydantic models for financial information extraction

class FinancialMetric(BaseModel):
    value: float
    unit: str
    year_over_year_change: Optional[float] = None
    quarter_over_quarter_change: Optional[float] = None

class CompanyFinancials(BaseModel):
    company_name: str
    ticker: str
    reporting_period: str
    fiscal_year: int
    fiscal_quarter: int
    
    revenue: FinancialMetric
    gross_profit: FinancialMetric
    operating_income: FinancialMetric
    net_income: FinancialMetric
    eps: FinancialMetric
    
    cash_and_equivalents: Optional[FinancialMetric] = None
    total_assets: Optional[FinancialMetric] = None
    total_liabilities: Optional[FinancialMetric] = None
    total_equity: Optional[FinancialMetric] = None
    
    key_highlights: List[str]
    risk_factors: Optional[List[str]] = None
    
    @validator('reporting_period')
    def validate_reporting_period(cls, v):
        if not re.match(r'Q[1-4] \d{4}|FY \d{4}', v):
            raise ValueError('Reporting period must be in format "Q1 2023" or "FY 2023"')
        return v

# Create a parser using the Pydantic model
parser = PydanticOutputParser(pydantic_object=CompanyFinancials)

# Print the schema for reference
print("JSON SCHEMA FOR FINANCIAL EXTRACTION:")
print("-" * 50)
print(json.dumps(CompanyFinancials.schema(), indent=2))

## 4. Implementing Structured Generation with LLMs

Now that we understand the principles of structured generation and have designed our schema, let's implement extraction from real financial documents. We'll use earnings reports as an example.

In [ ]:
# Sample earnings report excerpt
earnings_report = """
APPLE INC.
CONDENSED CONSOLIDATED STATEMENTS OF OPERATIONS (Unaudited)
(In millions, except number of shares which are reflected in thousands and per share amounts)

                                        Three Months Ended         Six Months Ended
                                   March 25, 2023  March 26, 2022  March 25, 2023  March 26, 2022
Net sales                               $94,836        $97,278       $212,239       $221,158
Cost of sales                            54,018         52,773        124,223        121,007
Gross margin                             40,818         44,505         88,016        100,151
Operating expenses:
  Research and development                7,461          6,387         14,472         12,594
  Selling, general and administrative     6,619          6,060         13,277         12,163
Total operating expenses                 14,080         12,447         27,749         24,757
Operating income                         26,738         32,058         60,267         75,394
Other income/(expense), net                 259           160            371            115
Income before provision for income taxes 26,997         32,218         60,638         75,509
Provision for income taxes                4,155          4,959          9,457         12,195
Net income                              $22,842        $27,259        $51,181        $63,314

Earnings per share:
  Basic                                   $1.45          $1.67          $3.23          $3.86
  Diluted                                 $1.44          $1.66          $3.22          $3.83
Shares used in computing earnings per share:
  Basic                                15,817,320     16,402,178     15,857,362     16,423,632
  Diluted                              15,906,539     16,571,096     15,947,728     16,571,138

PRESS RELEASE HIGHLIGHTS:
- Apple's board of directors has declared a cash dividend of $0.24 per share of the Company's common stock, an increase of 4 percent.
- The Company generated over $28 billion in operating cash flow and returned over $23 billion to shareholders during the quarter.
- iPhone revenue was $51.3 billion, down 1% year over year due to supply constraints.
- Services revenue reached an all-time record of $20.9 billion, up 5% year over year.
- Wearables, Home and Accessories revenue was $8.8 billion, down 1% year over year.
"""

# Create a prompt template that includes the schema
extraction_template = """
Extract the following financial information from the earnings report below. 
Format the output as a valid JSON object that follows this schema:

{schema}

Earnings Report:
{earnings_report}

JSON Output:
"""

# Set up the prompt
prompt = PromptTemplate(
    template=extraction_template,
    input_variables=["earnings_report"],
    partial_variables={"schema": parser.get_format_instructions()}
)

# Format the prompt with our earnings report
formatted_prompt = prompt.format(earnings_report=earnings_report)

# Create the LLM chain
llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)
extraction_chain = LLMChain(llm=llm, prompt=prompt)

# Run the extraction
response = extraction_chain.run(earnings_report=earnings_report)

# Parse and validate the extracted data
try:
    extracted_data = json.loads(response)
    validated_data = CompanyFinancials(**extracted_data)
    print("EXTRACTED FINANCIAL DATA:")
    print("-" * 50)
    print(json.dumps(validated_data.dict(), indent=2))
except Exception as e:
    print(f"Error validating extracted data: {e}")
    print("Raw response:")
    print(response)

## 5. Advanced Structured Generation Techniques

Now let's explore more advanced techniques for structured generation, including:

1. **Function Calling**: Using OpenAI's function calling capability for structured outputs
2. **Multiple Schema Extraction**: Extracting different types of information simultaneously
3. **Error Handling and Fallbacks**: Ensuring robust extraction even with imperfect inputs
4. **Iterative Refinement**: Improving extraction quality through multiple passes

In [ ]:
# Function calling for structured generation
def extract_with_function_calling(text):
    """Extract financial information using OpenAI's function calling capability"""
    
    # Define the function schema
    functions = [
        {
            "name": "extract_financial_data",
            "description": "Extract financial data from earnings reports and other financial documents",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company"
                    },
                    "ticker": {
                        "type": "string",
                        "description": "The stock ticker symbol"
                    },
                    "fiscal_period": {
                        "type": "object",
                        "properties": {
                            "quarter": {
                                "type": "integer",
                                "enum": [1, 2, 3, 4],
                                "description": "The fiscal quarter (1-4)"
                            },
                            "year": {
                                "type": "integer",
                                "description": "The fiscal year"
                            }
                        },
                        "required": ["quarter", "year"]
                    },
                    "financial_metrics": {
                        "type": "object",
                        "properties": {
                            "revenue": {
                                "type": "object",
                                "properties": {
                                    "value": {"type": "number"},
                                    "unit": {"type": "string"},
                                    "year_over_year_change": {"type": "number", "description": "Percentage change from previous year"}
                                },
                                "required": ["value", "unit"]
                            },
                            "net_income": {
                                "type": "object",
                                "properties": {
                                    "value": {"type": "number"},
                                    "unit": {"type": "string"},
                                    "year_over_year_change": {"type": "number", "description": "Percentage change from previous year"}
                                },
                                "required": ["value", "unit"]
                            },
                            "eps": {
                                "type": "object",
                                "properties": {
                                    "value": {"type": "number"},
                                    "unit": {"type": "string"},
                                    "year_over_year_change": {"type": "number", "description": "Percentage change from previous year"}
                                },
                                "required": ["value", "unit"]
                            }
                        },
                        "required": ["revenue", "net_income", "eps"]
                    },
                    "key_highlights": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "Main highlights or key points from the earnings report"
                    }
                },
                "required": ["company_name", "fiscal_period", "financial_metrics", "key_highlights"]
            }
        }
    ]
    
    # Call the model with function calling
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        messages=[
            {"role": "system", "content": "You are a financial data extraction assistant specialized in extracting structured information from earnings reports."},
            {"role": "user", "content": f"Extract the key financial information from this document:\n\n{text}"}
        ],
        functions=functions,
        function_call={"name": "extract_financial_data"}
    )
    
    function_call = response.choices[0].message.function_call
    
    if function_call:
        extracted_data = json.loads(function_call.arguments)
        return extracted_data
    else:
        return {"error": "Function was not called"}

# Test the function calling approach
function_calling_results = extract_with_function_calling(earnings_report)
print("FUNCTION CALLING EXTRACTION RESULTS:")
print("-" * 50)
print(json.dumps(function_calling_results, indent=2))

## 6. Multi-Document Extraction and Aggregation

In real-world financial applications, we often need to extract and aggregate information from multiple documents. Let's simulate this with quarterly reports and demonstrate how to build a comprehensive financial trend analysis.

In [ ]:
# Simulated quarterly reports for a year
quarterly_reports = {
    "Q1 2023": """
    ACME CORPORATION
    Q1 2023 Earnings Report
    
    Financial Highlights:
    - Revenue: $845 million, up 12% year-over-year
    - Net Income: $126 million, up 8% year-over-year
    - EPS: $1.28, compared to $1.19 in Q1 2022
    - Operating Margin: 18.2%
    
    Key Business Developments:
    - Launched new AI-powered financial analytics platform
    - Expanded operations in European markets with two new offices
    - Completed acquisition of FinTech startup DataFlow
    - R&D expenses increased by 15% to support innovation initiatives
    """,
    
    "Q2 2023": """
    ACME CORPORATION
    Q2 2023 Earnings Report
    
    Financial Highlights:
    - Revenue: $892 million, up 14% year-over-year
    - Net Income: $138 million, up 11% year-over-year
    - EPS: $1.40, compared to $1.26 in Q2 2022
    - Operating Margin: 19.1%
    
    Key Business Developments:
    - Secured three major enterprise clients for the new analytics platform
    - Announced strategic partnership with Global Financial Services
    - Share repurchase program of $200 million authorized
    - Cloud-based service subscriptions grew by 32%
    """,
    
    "Q3 2023": """
    ACME CORPORATION
    Q3 2023 Earnings Report
    
    Financial Highlights:
    - Revenue: $925 million, up 16% year-over-year
    - Net Income: $152 million, up 18% year-over-year
    - EPS: $1.55, compared to $1.31 in Q3 2022
    - Operating Margin: 20.3%
    
    Key Business Developments:
    - Crossed 1 million active users on the analytics platform
    - Opened new data center to support growing cloud operations
    - Increased quarterly dividend by 5% to $0.32 per share
    - Launched advanced risk management module for institutional clients
    """,
    
    "Q4 2023": """
    ACME CORPORATION
    Q4 2023 Earnings Report
    
    Financial Highlights:
    - Revenue: $986 million, up 18% year-over-year
    - Net Income: $167 million, up 15% year-over-year
    - EPS: $1.70, compared to $1.48 in Q4 2022
    - Operating Margin: 21.2%
    
    Key Business Developments:
    - Full year 2023 revenue reached $3.65 billion, up 15% from 2022
    - Annual recurring revenue from subscriptions surpassed $1 billion
    - Board approved expansion plan for Asia-Pacific region
    - Announced next-generation platform features coming in Q1 2024
    """
}

# Define a function to process multiple reports
def process_quarterly_reports(reports):
    results = {}
    
    for quarter, report in reports.items():
        print(f"Processing {quarter}...")
        
        # Extract data using function calling
        extracted_data = extract_with_function_calling(
            f"Quarterly Report for {quarter}:\n\n{report}"
        )
        
        # Store the results
        results[quarter] = extracted_data
    
    return results

# Process all quarterly reports
quarterly_results = process_quarterly_reports(quarterly_reports)

# Create a financial trend analysis
def generate_trend_analysis(quarterly_data):
    # Extract revenue, net income, and EPS for each quarter
    quarters = sorted(quarterly_data.keys())
    revenue = [quarterly_data[q]["financial_metrics"]["revenue"]["value"] for q in quarters]
    net_income = [quarterly_data[q]["financial_metrics"]["net_income"]["value"] for q in quarters]
    eps = [quarterly_data[q]["financial_metrics"]["eps"]["value"] for q in quarters]
    
    # Create DataFrame for analysis
    df = pd.DataFrame({
        'Quarter': quarters,
        'Revenue (millions)': revenue,
        'Net Income (millions)': net_income,
        'EPS': eps
    })
    
    # Calculate quarter-over-quarter growth
    df['Revenue QoQ Growth'] = df['Revenue (millions)'].pct_change() * 100
    df['Net Income QoQ Growth'] = df['Net Income (millions)'].pct_change() * 100
    df['EPS QoQ Growth'] = df['EPS'].pct_change() * 100
    
    return df

# Generate and display the trend analysis
trend_df = generate_trend_analysis(quarterly_results)
print("\nQUARTERLY FINANCIAL TRENDS:")
print("-" * 50)
print(trend_df)

# Visualize the trends
plt.figure(figsize=(14, 8))

# Revenue and Net Income on left y-axis
ax1 = plt.subplot()
ax1.plot(trend_df['Quarter'], trend_df['Revenue (millions)'], 'b-o', label='Revenue')
ax1.plot(trend_df['Quarter'], trend_df['Net Income (millions)'], 'g-o', label='Net Income')
ax1.set_xlabel('Quarter')
ax1.set_ylabel('Amount ($ millions)')
ax1.tick_params(axis='y', labelcolor='b')
ax1.legend(loc='upper left')

# EPS on right y-axis
ax2 = ax1.twinx()
ax2.plot(trend_df['Quarter'], trend_df['EPS'], 'r-o', label='EPS')
ax2.set_ylabel('Earnings Per Share ($)')
ax2.tick_params(axis='y', labelcolor='r')
ax2.legend(loc='upper right')

plt.title('ACME Corporation - 2023 Quarterly Financial Performance')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Structured Information Retrieval

One of the most powerful applications of structured generation is enabling precise information retrieval from large volumes of financial documents. In this section, we'll demonstrate how to build a queryable database of structured financial information.

In [ ]:
# Create a simulated database of structured financial information
financial_database = {}

# Process our quarterly reports into the database
for quarter, report in quarterly_reports.items():
    company = "ACME Corporation"
    period = quarter
    
    # Add to our database if the company doesn't exist
    if company not in financial_database:
        financial_database[company] = {}
    
    # Extract the data
    extracted_data = extract_with_function_calling(
        f"Quarterly Report for {quarter}:\n\n{report}"
    )
    
    # Store in our database
    financial_database[company][period] = extracted_data

# Function to query the database
def query_financial_database(query):
    """
    Natural language interface to query the financial database
    """
    # Define the function for querying
    functions = [
        {
            "name": "search_financial_database",
            "description": "Search for specific financial information in the database",
            "parameters": {
                "type": "object",
                "properties": {
                    "company": {
                        "type": "string",
                        "description": "The company to search for"
                    },
                    "time_period": {
                        "type": "string",
                        "description": "The time period to search for (e.g., 'Q1 2023', 'Q2 2023')"
                    },
                    "metric": {
                        "type": "string",
                        "description": "The financial metric to retrieve (e.g., 'revenue', 'net_income', 'eps')"
                    },
                    "comparison": {
                        "type": "string",
                        "enum": ["latest", "highest", "lowest", "trend", "specific"],
                        "description": "The type of comparison or analysis to perform"
                    }
                },
                "required": ["company"]
            }
        }
    ]
    
    # Call the model to parse the query
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a financial database query assistant. Parse the user's query into structured parameters."},
            {"role": "user", "content": query}
        ],
        functions=functions,
        function_call={"name": "search_financial_database"}
    )
    
    # Extract the parsed query
    function_call = response.choices[0].message.function_call
    parsed_query = json.loads(function_call.arguments)
    
    # Execute the query on our database
    results = execute_financial_query(parsed_query)
    return results

def execute_financial_query(parsed_query):
    """Execute the parsed query on our financial database"""
    company = parsed_query.get("company")
    period = parsed_query.get("time_period")
    metric = parsed_query.get("metric")
    comparison = parsed_query.get("comparison", "specific")
    
    # Check if company exists
    if company not in financial_database:
        return {"error": f"Company '{company}' not found in database"}
    
    company_data = financial_database[company]
    
    # Handle different comparison types
    if comparison == "latest":
        latest_period = sorted(company_data.keys())[-1]
        if metric:
            if metric == "revenue":
                return {
                    "company": company,
                    "period": latest_period,
                    "metric": metric,
                    "value": company_data[latest_period]["financial_metrics"]["revenue"]["value"],
                    "unit": company_data[latest_period]["financial_metrics"]["revenue"]["unit"]
                }
            elif metric == "net_income":
                return {
                    "company": company,
                    "period": latest_period,
                    "metric": metric,
                    "value": company_data[latest_period]["financial_metrics"]["net_income"]["value"],
                    "unit": company_data[latest_period]["financial_metrics"]["net_income"]["unit"]
                }
            elif metric == "eps":
                return {
                    "company": company,
                    "period": latest_period,
                    "metric": metric,
                    "value": company_data[latest_period]["financial_metrics"]["eps"]["value"],
                    "unit": company_data[latest_period]["financial_metrics"]["eps"]["unit"]
                }
            else:
                return {"error": f"Metric '{metric}' not found"}
        else:
            return company_data[latest_period]
            
    elif comparison == "trend":
        if not metric:
            return {"error": "Metric is required for trend analysis"}
            
        periods = sorted(company_data.keys())
        values = []
        
        for p in periods:
            if metric == "revenue":
                values.append({
                    "period": p,
                    "value": company_data[p]["financial_metrics"]["revenue"]["value"],
                    "unit": company_data[p]["financial_metrics"]["revenue"]["unit"]
                })
            elif metric == "net_income":
                values.append({
                    "period": p,
                    "value": company_data[p]["financial_metrics"]["net_income"]["value"],
                    "unit": company_data[p]["financial_metrics"]["net_income"]["unit"]
                })
            elif metric == "eps":
                values.append({
                    "period": p,
                    "value": company_data[p]["financial_metrics"]["eps"]["value"],
                    "unit": company_data[p]["financial_metrics"]["eps"]["unit"]
                })
        
        return {
            "company": company,
            "metric": metric,
            "trend": values
        }
        
    elif comparison == "specific":
        if not period:
            return {"error": "Time period is required for specific queries"}
        if period not in company_data:
            return {"error": f"Period '{period}' not found for company '{company}'"}
            
        if metric:
            if metric == "revenue":
                return {
                    "company": company,
                    "period": period,
                    "metric": metric,
                    "value": company_data[period]["financial_metrics"]["revenue"]["value"],
                    "unit": company_data[period]["financial_metrics"]["revenue"]["unit"]
                }
            elif metric == "net_income":
                return {
                    "company": company,
                    "period": period,
                    "metric": metric,
                    "value": company_data[period]["financial_metrics"]["net_income"]["value"],
                    "unit": company_data[period]["financial_metrics"]["net_income"]["unit"]
                }
            elif metric == "eps":
                return {
                    "company": company,
                    "period": period,
                    "metric": metric,
                    "value": company_data[period]["financial_metrics"]["eps"]["value"],
                    "unit": company_data[period]["financial_metrics"]["eps"]["unit"]
                }
            else:
                return {"error": f"Metric '{metric}' not found"}
        else:
            return company_data[period]
    
    return {"error": "Unsupported query type"}

# Test the query system with various questions
test_queries = [
    "What was ACME Corporation's revenue in Q3 2023?",
    "Show me the EPS trend for ACME Corporation throughout 2023",
    "What was ACME's latest reported net income?",
    "Give me all financial metrics for ACME in Q2 2023"
]

for query in test_queries:
    print(f"\nQUERY: {query}")
    print("-" * 50)
    result = query_financial_database(query)
    print(json.dumps(result, indent=2))

## 8. Case Study: Financial Sentiment Analysis with Structured Output

Let's apply structured generation to analyze sentiment in financial news articles and earnings call transcripts. This demonstrates how combining LLMs with structured outputs can yield powerful financial analytics.

In [ ]:
# Sample financial news and earnings call transcripts
financial_texts = [
    {
        "type": "news_article",
        "source": "Financial Times",
        "title": "ACME Reports Strong Q4 Results, Beats Analyst Expectations",
        "content": """
        ACME Corporation reported fourth-quarter results that exceeded Wall Street expectations on Thursday, 
        driven by strong performance in its software division and better-than-expected margins.
        
        The technology company reported revenue of $986 million, up 18% year-over-year, and earnings per share of $1.70, 
        compared with analyst expectations of $950 million and $1.65 per share respectively.
        
        "Our results demonstrate the strength of our business model and our ability to execute on our strategic priorities," 
        said Jane Smith, CEO of ACME Corporation. "We're seeing strong demand across all our product lines."
        
        The company also provided an optimistic outlook for 2024, projecting revenue growth of 15-17% and margin expansion 
        of approximately 100 basis points. ACME's shares rose 7% in after-hours trading following the announcement.
        
        However, some analysts have expressed concerns about increasing competition in the company's core markets and 
        potential macroeconomic headwinds that could impact enterprise spending in the coming year.
        """
    },
    {
        "type": "earnings_call",
        "company": "TechFin Solutions",
        "quarter": "Q1 2023",
        "content": """
        Operator: Good afternoon and welcome to TechFin Solutions' first quarter 2023 earnings call. 
        At this time, all participants are in listen-only mode. After the speakers' presentation, 
        there will be a question-and-answer session.
        
        CEO: Thank you, operator. Good afternoon everyone and thank you for joining us today. 
        I'm disappointed to report that our first quarter results fell short of our expectations. 
        Revenue came in at $342 million, down 5% year-over-year, and significantly below our guidance 
        of $380-400 million. Our adjusted earnings per share was $0.82, compared to $1.05 in the 
        prior year period.
        
        Several factors contributed to our underperformance. First, we experienced unexpected delays 
        in closing several large enterprise deals, which have since pushed into the second quarter. 
        Second, our international expansion has been slower than anticipated due to regulatory challenges 
        in key markets. Finally, we've seen increased competitive pressure in our core banking software segment.
        
        Despite these challenges, there are bright spots in our results. Our recurring revenue grew 12%, 
        and customer retention remained strong at 94%. We've also made progress on our cost optimization 
        initiatives, which we expect will yield approximately $25 million in annual savings.
        
        Looking ahead, we're revising our full-year guidance. We now expect revenue growth of 2-4%, 
        down from our previous projection of 8-10%. We're taking decisive actions to address our challenges, 
        including accelerating our product roadmap and restructuring our sales organization.
        """
    }
]

# Define a schema for sentiment analysis
class EntitySentiment(BaseModel):
    entity: str
    sentiment: str  # positive, negative, neutral
    confidence: float  # 0-1
    relevant_quotes: List[str]

class FinancialMetricSentiment(BaseModel):
    metric: str
    performance: str  # exceeds, meets, misses expectations
    sentiment: str  # positive, negative, neutral
    value: Optional[str] = None
    comparison: Optional[str] = None  # year-over-year, quarter-over-quarter, vs expectations

class FinancialSentimentAnalysis(BaseModel):
    document_type: str  # news_article, earnings_call, analyst_report
    overall_sentiment: str  # positive, negative, neutral, mixed
    sentiment_score: float  # -1 to 1
    key_entities: List[EntitySentiment]
    financial_metrics: List[FinancialMetricSentiment]
    forward_looking_statements: List[str]
    risk_factors: List[str]
    confidence_level: float  # 0-1

# Function to analyze sentiment with structured output
def analyze_financial_sentiment(text, document_type):
    """Analyze sentiment in financial texts using structured generation"""
    
    # Create a system message with the task description
    system_message = """
    You are a financial sentiment analysis expert. Analyze the provided financial text and extract sentiment information
    in a structured format. Focus on overall sentiment, entity-level sentiment, financial metrics, forward-looking statements,
    and risk factors. Provide confidence scores where appropriate.
    """
    
    # Create user message with the text and schema
    user_message = f"""
    Analyze the sentiment in the following {document_type}:
    
    {text}
    
    Provide your analysis in the following JSON structure:
    
    {{
      "document_type": "{document_type}",
      "overall_sentiment": "positive/negative/neutral/mixed",
      "sentiment_score": <float between -1 and 1>,
      "key_entities": [
        {{
          "entity": <entity name>,
          "sentiment": "positive/negative/neutral",
          "confidence": <float between 0 and 1>,
          "relevant_quotes": [<direct quotes from text>]
        }}
      ],
      "financial_metrics": [
        {{
          "metric": <metric name>,
          "performance": "exceeds/meets/misses expectations",
          "sentiment": "positive/negative/neutral",
          "value": <optional value>,
          "comparison": <optional comparison>
        }}
      ],
      "forward_looking_statements": [<list of forward-looking statements>],
      "risk_factors": [<list of risk factors or challenges mentioned>],
      "confidence_level": <float between 0 and 1>
    }}
    """
    
    # Call the model
    response = openai_client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        response_format={"type": "json_object"}
    )
    
    # Parse the response
    analysis = json.loads(response.choices[0].message.content)
    
    # Validate against our schema
    try:
        validated_analysis = FinancialSentimentAnalysis(**analysis)
        return validated_analysis.dict()
    except Exception as e:
        print(f"Validation error: {e}")
        return analysis

# Analyze sentiment for each document
for i, doc in enumerate(financial_texts):
    print(f"\nANALYZING DOCUMENT {i+1}: {doc['type']}")
    print("-" * 50)
    
    sentiment_analysis = analyze_financial_sentiment(doc['content'], doc['type'])
    
    print(f"Overall Sentiment: {sentiment_analysis['overall_sentiment']}")
    print(f"Sentiment Score: {sentiment_analysis['sentiment_score']}")
    
    print("\nKey Entities:")
    for entity in sentiment_analysis['key_entities']:
        print(f"- {entity['entity']}: {entity['sentiment']} (confidence: {entity['confidence']})")
    
    print("\nFinancial Metrics:")
    for metric in sentiment_analysis['financial_metrics']:
        print(f"- {metric['metric']}: {metric['performance']} ({metric['sentiment']})")
    
    print("\nForward-Looking Statements:")
    for statement in sentiment_analysis['forward_looking_statements'][:3]:
        print(f"- {statement}")
    
    print("\nRisk Factors:")
    for risk in sentiment_analysis['risk_factors'][:3]:
        print(f"- {risk}")

## 9. Best Practices for Structured Generation in Finance

Based on our exploration, here are key best practices for implementing structured generation in financial applications:

### Schema Design:
- **Start Simple**: Begin with a minimal schema and expand as needed
- **Use Nested Structures**: Organize related information hierarchically
- **Include Validation Rules**: Define constraints to ensure data quality
- **Document Fields Clearly**: Provide clear descriptions for each field

### Prompt Engineering:
- **Be Explicit**: Clearly state the format requirements in your prompts
- **Provide Examples**: Include sample outputs to guide the model
- **Use Temperature Control**: Lower temperature (0-0.3) for more deterministic outputs
- **Use System Messages**: Set the context with system messages

### Error Handling:
- **Validate Outputs**: Always validate generated structures against your schema
- **Implement Fallbacks**: Have mechanisms to handle parsing failures
- **Consider Multiple Passes**: Use iterative refinement for complex extractions

### Practical Implementation:
- **Use Function Calling**: Leverage OpenAI's function calling for complex schemas
- **Batch Related Extractions**: Process similar documents together
- **Consider Trade-offs**: Balance between structure complexity and extraction reliability
- **Build Evaluation Metrics**: Track accuracy and completeness of extractions

## 10. Practical Exercise: Build Your Own Financial Information Extractor

Now it's your turn to practice structured generation! In this exercise, you'll build a custom financial information extractor for a specific type of financial document.

### Exercise Steps:
1. Choose a financial document type (earnings report, analyst report, SEC filing, etc.)
2. Design a schema for the information you want to extract
3. Create a prompt template with your schema
4. Test your extractor on a sample document
5. Refine your approach based on the results

Use the code patterns demonstrated in this notebook as a starting point. Here's a template to get you started:

In [ ]:
# Exercise template

# 1. Define your Pydantic model (schema)
class MyFinancialSchema(BaseModel):
    # Define your schema here
    pass

# 2. Create a sample document
sample_document = """
# Paste your sample financial document here
"""

# 3. Create your extraction function
def extract_financial_information(document, schema_class):
    # Implement your extraction function
    pass

# 4. Test your extractor
results = extract_financial_information(sample_document, MyFinancialSchema)
print(json.dumps(results, indent=2))

# 5. Evaluate and refine
# How accurate were your results? What improvements could you make?

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.

## 11. Conclusion

Structured generation is a powerful technique for financial information extraction and analysis. By combining the linguistic capabilities of LLMs with structured output formats, financial professionals can:

1. **Automate document processing**: Extract key information from financial documents at scale
2. **Standardize information**: Convert unstructured text into consistent, structured formats
3. **Enable advanced analytics**: Build queryable databases of financial information
4. **Improve decision-making**: Generate insights from large volumes of financial text

As LLMs continue to evolve, structured generation will become an increasingly important tool in the financial professional's toolkit, enabling more efficient information processing and more sophisticated financial analyses.

In the next notebook, we'll explore multimodal AI systems through the lens of text-to-image models, providing another perspective on the future of AI in finance.